In [1]:
from __future__ import division, print_function, absolute_import

In [2]:
import tflearn
from tflearn.data_utils import shuffle
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.estimator import regression
from tflearn.data_preprocessing import ImagePreprocessing
from tflearn.data_augmentation import ImageAugmentation
import pickle

/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
X, Y, X_test, Y_test = pickle.load(open("full_dataset.pkl", "rb"), encoding="bytes")

In [4]:
X, Y = shuffle(X, Y)

In [5]:
img_prep = ImagePreprocessing()
img_prep.add_featurewise_zero_center()
img_prep.add_featurewise_stdnorm()

In [6]:
img_aug = ImageAugmentation()
img_aug.add_random_flip_leftright()
img_aug.add_random_rotation(max_angle=25.)
img_aug.add_random_blur(sigma_max=3.)

In [7]:
network = input_data(shape=[None, 32, 32, 3],
                     data_preprocessing=img_prep,
                     data_augmentation=img_aug)

In [8]:
network = conv_2d(network, 32, 3, activation='relu')

Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.


In [9]:
network = max_pool_2d(network, 2)

In [10]:
network = conv_2d(network, 64, 3, activation='relu')

In [11]:
network = conv_2d(network, 64, 3, activation='relu')

In [12]:
network = max_pool_2d(network, 2)

In [13]:
# network = fully_connected(network, 512, activation='relu')
network = dropout(network, 0.5)

In [14]:
# network = dropout(network, 0.5)
network = fully_connected(network, 512, activation='relu')

In [15]:
network = fully_connected(network, 2, activation='softmax')

In [16]:
network = regression(network, optimizer='adam',
                     loss='categorical_crossentropy',
                     learning_rate=0.001)

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [17]:
model = tflearn.DNN(network, tensorboard_verbose=0, checkpoint_path='bird-classifier.tfl.ckpt')

In [18]:
model.fit(X, Y, n_epoch=10, shuffle=True, validation_set=(X_test, Y_test),
          show_metric=True, batch_size=96,
          snapshot_epoch=True,
          run_id='bird-classifier')

Training Step: 5919  | total loss: 0.27714 | time: 108.360s
| Adam | epoch: 010 | loss: 0.27714 - acc: 0.8870 -- iter: 56736/56780
Training Step: 5920  | total loss: 0.28130 | time: 116.507s
| Adam | epoch: 010 | loss: 0.28130 - acc: 0.8869 | val_loss: 0.20189 - val_acc: 0.9211 -- iter: 56780/56780
--
INFO:tensorflow:/Users/namrata/PycharmProjects/bird-classifier.tfl.ckpt-5920 is not in all_model_checkpoint_paths. Manually adding it.


In [19]:
model.save("bird-classifier.tfl")
print("Network trained and saved as bird-classifier.tfl!")

INFO:tensorflow:/Users/namrata/PycharmProjects/bird-classifier.tfl is not in all_model_checkpoint_paths. Manually adding it.
Network trained and saved as bird-classifier.tfl!


In [20]:
import scipy

In [21]:
import numpy as np

In [22]:
from keras.preprocessing import image

Using TensorFlow backend.


In [23]:
#print()

In [24]:
model.load("bird-classifier.tfl")

INFO:tensorflow:Restoring parameters from /Users/namrata/PycharmProjects/bird-classifier.tfl


In [25]:
# Load the image file
img = scipy.ndimage.imread("birdtest1.bmp", mode="RGB")

# Scale it to 32x32
img = scipy.misc.imresize(img, (32, 32), interp="bicubic").astype(np.float32, casting='unsafe')

# # Predict
prediction = model.predict([img])

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0.
Use ``matplotlib.pyplot.imread`` instead.
  
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  """


In [26]:
# # Check the result.
is_bird = np.argmax(prediction[0]) == 1

In [27]:
if is_bird:
    print("That's a bird!")
else:
    print("That's not a bird!")

That's a bird!
